# Feature Engineering

This notebook creates the training dataset by merging three real datasets:
1. **Historical Crop Performance**: Actual yields per crop-province-year
2. **Soil Test Data**: Real farmer soil conditions (NPK, pH) aggregated by province
3. **Climate Data**: 5-year averages (2020-2024) for temperature, rainfall, humidity

Then:
- Extracts 9 features for each crop-province-soil combination
- Creates target suitability scores from actual historical yields
- Uses REAL data instead of simulated values


In [7]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path

# Add app to path to import modules
project_root = Path("../").resolve()
sys.path.append(str(project_root))

from app.services.data_loader import DataLoader
from app.services.feature_extractor import FeatureExtractor
from app.utils.data_processor import normalize_npk_level, parse_ph_range, parse_soil_types

# Initialize with explicit data directory path
data_dir = project_root / "raw_datasets"
data_loader = DataLoader(data_dir=str(data_dir))
data_loader.load_all_data()
feature_extractor = FeatureExtractor()

print("Data loaded successfully!")


Loading datasets...
Loaded 104 crop climate requirements
Loaded 104 crop requirements
Loaded 104 NPK requirements
Loaded 93369 historical performance records
Loading climate data (this may take a moment)...
Loaded 215556 climate records
Creating unified crop database...
Unified database created with 104 crops
Data loaded successfully!


In [8]:
# Load historical performance and calculate yields
historical_perf = pd.read_csv("../raw_datasets/historical_crop_performance.csv")
historical_perf['yield_per_ha'] = (
    historical_perf['Volume_Production'] / historical_perf['Area_Planted_Harvested']
)

# Clean data
historical_perf_clean = historical_perf[
    (historical_perf['yield_per_ha'].notna()) & 
    (historical_perf['yield_per_ha'] != float('inf')) &
    (historical_perf['yield_per_ha'] > 0)
].copy()

print(f"Valid yield records: {len(historical_perf_clean)}")


Valid yield records: 93369


In [9]:
# Load and aggregate soil test data
print("Loading soil test data...")
soil_test_df = pd.read_csv("../raw_datasets/soil_test_data.csv")
print(f"Loaded {len(soil_test_df)} soil test records")

# Normalize province names for matching
soil_test_df['province_normalized'] = soil_test_df['province'].str.strip().str.title()

# Aggregate soil data by province (get most common values)
def get_mode_or_default(series, default='Medium'):
    """Get mode value or return default if empty."""
    mode_values = series.mode()
    return mode_values[0] if len(mode_values) > 0 else default

province_soil_agg = soil_test_df.groupby('province_normalized').agg({
    'nitrogen': lambda x: get_mode_or_default(x, 'Medium'),
    'phosphorus': lambda x: get_mode_or_default(x, 'Medium'),
    'potassium': lambda x: get_mode_or_default(x, 'Medium'),
    'ph_min': 'mean',
    'ph_max': 'mean'
}).reset_index()
province_soil_agg.columns = ['province', 'nitrogen', 'phosphorus', 'potassium', 'ph_min', 'ph_max']

print(f"Aggregated soil data for {len(province_soil_agg)} provinces")

# Normalize province names in historical data for merging
historical_perf_clean['Province_normalized'] = historical_perf_clean['Province'].str.strip().str.title()

# Merge historical performance with soil data
print("\nMerging historical data with soil data...")
merged_data = historical_perf_clean.merge(
    province_soil_agg,
    left_on='Province_normalized',
    right_on='province',
    how='inner'  # Only keep records with soil data
)

print(f"Total merged records: {len(merged_data)}")
print(f"Records with soil data: {merged_data['nitrogen'].notna().sum()}")
print(f"Unique crop-province combinations with soil data: {merged_data[['Crop', 'Province']].drop_duplicates().shape[0]}")


Loading soil test data...
Loaded 22242 soil test records
Aggregated soil data for 62 provinces

Merging historical data with soil data...
Total merged records: 65898
Records with soil data: 65898
Unique crop-province combinations with soil data: 3935


In [10]:
# Create training dataset using merged data
training_data = []

# Sample a subset for faster processing (adjust as needed)
sample_size = min(5000, len(merged_data))
merged_sample = merged_data.sample(n=sample_size, random_state=42)

print(f"Processing {len(merged_sample)} records with real soil, climate, and yield data...")

for idx, row in merged_sample.iterrows():
    crop_name = row['Crop']
    province = row['Province']
    yield_per_ha = row['yield_per_ha']
    
    try:
        # Get crop data
        crop_data = data_loader.get_crop_by_name(crop_name)
        if crop_data is None:
            continue
        
        # Get climate averages for province
        try:
            climate = data_loader.get_climate_averages(province, None)
            if not all([climate['temperature'], climate['rainfall'], climate['humidity']]):
                continue
        except Exception as e:
            continue
        
        # Use REAL soil data from merged dataset
        farmer_nitrogen = str(row['nitrogen'])
        farmer_phosphorus = str(row['phosphorus'])
        farmer_potassium = str(row['potassium'])
        farmer_ph_min = float(row['ph_min'])
        farmer_ph_max = float(row['ph_max'])
        
        # Soil type - use common type (most common in Philippines)
        # Could be improved by sampling from crop's acceptable types
        farmer_soil_type = "Loam"
        
        # Get historical yield data for feature extraction (for regional success rate)
        hist_data = data_loader.get_historical_yield(crop_name, province)
        
        # Extract features using REAL data
        features = feature_extractor.extract_features(
            crop_data=crop_data,
            farmer_nitrogen=farmer_nitrogen,
            farmer_phosphorus=farmer_phosphorus,
            farmer_potassium=farmer_potassium,
            farmer_ph_min=farmer_ph_min,
            farmer_ph_max=farmer_ph_max,
            farmer_soil_type=farmer_soil_type,
            avg_temperature=climate['temperature'],
            avg_rainfall=climate['rainfall'],
            avg_humidity=climate['humidity'],
            historical_yield_data=hist_data,  # For regional success features
            current_month=6  # June (can be varied)
        )
        
        # Calculate target suitability score from ACTUAL yield
        # Normalize yield to 0-100 score (assume max expected yield is 20 tons/ha)
        suitability_score = min(100.0, max(0.0, (yield_per_ha / 20.0) * 80.0 + 20.0))
        
        # Add to training data
        training_row = features.copy()
        training_row['suitability_score'] = suitability_score
        training_row['crop_name'] = crop_name
        training_row['province'] = province
        training_row['category'] = crop_data.get('Category', 'Unknown')
        training_row['actual_yield'] = yield_per_ha  # Keep actual yield for reference
        
        training_data.append(training_row)
        
    except Exception as e:
        if len(training_data) % 1000 == 0:  # Only print errors occasionally
            print(f"Error processing {crop_name} in {province}: {e}")
        continue
    
    if len(training_data) % 500 == 0:
        print(f"Processed {len(training_data)} records...")

print(f"\nTotal training records created: {len(training_data)}")
print(f"Records with valid data: {len([r for r in training_data if r.get('suitability_score', 0) > 0])}")


Processing 5000 records with real soil, climate, and yield data...
Processed 500 records...
Processed 1000 records...
Processed 1500 records...
Processed 2000 records...
Processed 2500 records...
Processed 3000 records...
Processed 3500 records...
Processed 4000 records...
Processed 4500 records...

Total training records created: 4884
Records with valid data: 4884


In [12]:
# Convert to DataFrame
train_df = pd.DataFrame(training_data)

print("Training dataset shape:", train_df.shape)
print("\nColumns:", train_df.columns.tolist())
print("\nFirst few rows:")
print(train_df.head())
print("\nTarget variable (suitability_score) statistics:")
print(train_df['suitability_score'].describe())

# Save training dataset
train_df.to_csv("../models/training_dataset.csv", index=False)
print("\nTraining dataset saved to models/training_dataset.csv")


Training dataset shape: (4884, 14)

Columns: ['npk_match', 'ph_proximity', 'temp_suitability', 'rainfall_suitability', 'humidity_suitability', 'soil_match', 'historical_yield', 'season_alignment', 'regional_success', 'suitability_score', 'crop_name', 'province', 'category', 'actual_yield']

First few rows:
   npk_match  ph_proximity  temp_suitability  rainfall_suitability  \
0   0.000000      0.801245          0.956212              0.510209   
1   0.000000      0.547619          0.780627              0.000000   
2   0.666667      0.819444          0.607525              0.124888   
3   0.666667      0.802083          0.913970              0.000000   
4   0.666667      0.895717          0.729678              0.000000   

   humidity_suitability  soil_match  historical_yield  season_alignment  \
0              0.941970         0.3          1.000000               1.0   
1              0.821305         1.0          0.416671               1.0   
2              0.782908         1.0          1